In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
import time
from random import randint
import re


In [ ]:
def gettingUrl(url):

    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    # print(userAgent)
    options.headless = True  # setting this to true makes the browser work in the background
    options.add_argument("--proxy-server='direct://'")
    options.add_argument("--proxy-bypass-list=*")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-gpu")
    options.add_argument('--lang=en_US')
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--disable-extensions")
    options.add_argument(f'user-agent={userAgent}')
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    # DRIVER_PATH = "/usr/bin/chromedriver"
    driver = webdriver.Chrome(service=Service(
        ChromeDriverManager().install()), options=options)
    driver.get(url)
    driver.get_screenshot_as_file("screenshot1.png")
    time.sleep(randint(10,30))

    html = driver.page_source
    # this renders the JS code and stores all of this information in static HTMl code.
    return html


In [ ]:
# extracting data from acs nano site
html = gettingUrl("https://pubs.acs.org/doi/10.1021/acsnano.5b05040")


In [ ]:
# finding out the number of times the research paper number 5 is mentioned in the text.
# creating an instance of the BeautifulSoup library
soup_new = BeautifulSoup(html, 'lxml')
a_tags = soup_new.find('div', class_="article_content-left ui-resizable")
# print(a_tags)
count = a_tags.find_all('a', class_="ref5")
print(len(count))

In [ ]:
#to find the last author names of the references
soup_new = BeautifulSoup(html, 'lxml')
allLastAuthors =[]

relevant = soup_new.find('div', class_="article_content-left ui-resizable")
string_Refs = relevant.find('p', class_="references-count")
string = string_Refs.text
number_Refs = re.findall(r'\d+', string)

print(number_Refs[0])

i=0

for i in range(1, int(number_Refs[0])+1):

    info = relevant.find('li', {"id":"ref"+str(i)})

    if info is None:
        allLastAuthors.append("No information")
    # print(info)

    authors = info.find_all('span',class_='NLM_contrib-group')
    # print(authors)
    last_author = authors[-1].text
    allLastAuthors.append(last_author)
    i+=1

# print((allLastAuthors))

In [ ]:
# creating an instance of the BeautifulSoup library
soup2 = BeautifulSoup(html, 'lxml')
links = soup2.find('div', class_="article_content-left ui-resizable")
# creating a list containing the google scholar links for all the reference papers.
paperLinksgoogleScholar = links.find_all('a', class_="google-scholar")
scholarLinks = []
for link in paperLinksgoogleScholar:
    # A new list containing the google scholar links for only the first three papers is created.
    scholarLinks.append(link['href'])
for link in scholarLinks:
    index = scholarLinks.index(link)
    print(index+1, link)


In [ ]:
authors_new = []
for link in scholarLinks:
    html = gettingUrl(link)
    # creating an instance of the BeautifulSoup library
    soup3 = BeautifulSoup(html, 'lxml')
    content = soup3.find('div', class_="gs_a")

    if content == None:
        print("No Data Found")
    else:
        authors = content.find_all('a')

        # getting the name of the professor to then set up further relationships.
        print(authors[-1].text)
        authors_new.append(authors[-1].text)

In [ ]:
# to find the publication house name of the referenced paper from google scholar.

name = content.text
# print(name)

if "science" in name:
    print("science")
elif 'Wiley' in name:
    print("wiley")
elif 'nature' in name:
    print("nature")
elif 'sciencedirect' in name:
    print("sciencedirect")
elif 'Springer' in name:
    print("Springer")
elif 'Elsevier' in name:
    print("Elsevier")
elif 'mdpi' in name:
    print("mdpi")
elif 'ACS Publications' in name:
    print("ACS Publications")
elif 'ieee' in name:
    print("ieee")
else:
    print("other")


In [ ]:
# finding out the publication house link of the paper from google scholar.
paperLink = soup3.find('h3', class_='gs_rt')
print(paperLink.a['href'])


In [ ]:
# finding out the name of the research paper by extracting the text from the google scholar page.

options = Options()
ua = UserAgent
userAgent = ua
options.headless = True
options.add_argument("--proxy-server='direct://'")
options.add_argument("--proxy-bypass-list=*")
options.add_argument("--start-maximized")
options.add_argument("--disable-gpu")
options.add_argument('--lang=en_US')
options.add_argument("--window-size=1920,1080")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--disable-extensions")
options.add_argument(f'user-agent={userAgent}')
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(
    ChromeDriverManager().install()), options=options)
driver.get(scholarLinks[4])
# driver.find_element(By.XPATH, value='//*[@id="gs_res_ccl_mid"]/div/div[2]/div[3]/a[2]').click()
driver.find_element(By.LINK_TEXT, "Cite").click()
time.sleep(randint(10, 30))

# To get the name of the 5th paper
html = gettingUrl(driver.current_url)
# print(driver.current_url)
soup4 = BeautifulSoup(html, 'lxml')
paperName = soup4.find('div', class_='gs_citr')
print(paperName.i.text)  # printing the name of the 5th paper.

# quits the whole browser session along with all the associated browser windows, tabs and pop-ups.
driver.quit()


In [ ]:
# Extraction from "Nature" publication house.
html = gettingUrl("https://www.nature.com/articles/s41417-022-00495-w")


# to get the google scholar links of the references.
paperLinksgoogleScholarnature = []
# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'html.parser')
links = soup.find('ol', class_="c-article-references")
tags = links.find_all('a', {'data-track-action': 'google scholar reference'})
for tag in tags:
    # creating a list containing the google scholar links for all the
    paperLinksgoogleScholarnature.append(tag['href'])


# to get the doi of the references.
natureDOI = []
refNumber = len(paperLinksgoogleScholarnature)

i=0

for i in range(1, int(refNumber)+1):
    doiLinks = links.find('a', {'aria-label': 'Article reference '+str(i)})

    if doiLinks == None:
        natureDOI.append("No DOI")
    else:
        natureDOI.append(doiLinks['href'])
    i+=1


# extracting the authors from the main page itself.
text = soup.find_all('ol', class_='c-article-references')
# print(text)
text_main = soup.find_all(
    'li', class_='c-article-references__item js-c-reading-companion-references-item')
# print(text_main[0])

Titles = []
allLastAuthors_nature = []
journalName = []
yearPublication = []

for ref in text_main:
    string = ref.find('p', class_='c-article-references__text')

    togetdeets = string.text.split('.')

    # returns authors in the form of a string.
    allAuthors = togetdeets[0]
    allAuthorsList = allAuthors.split(',')  # converting to a list

    Titles.append(togetdeets[1])  # gets the title of the references
    
    journalName.append(togetdeets[2])# gets the journal name of the references

    if len(togetdeets) < 4:
        yearPublication.append(' ') #gets the year of publication of the references
    else:
        year = togetdeets[3].split(';')
        yearPublication.append(year[0])

    if allAuthorsList[-1] == ' et al':
        allLastAuthors_nature.append(allAuthorsList[-2])
    else:
        allLastAuthors_nature.append(allAuthorsList[-1])

print(yearPublication)

In [ ]:
# Extraction from "Springer" publication house.
html = gettingUrl("https://link.springer.com/article/10.1007/s43673-022-00064-1")
paperLinksgoogleScholarspringer = []
# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'lxml')
links = soup.find('ol', class_="c-article-references")
refNumber = len(links)
k = 0
for k in range(1, int(refNumber)+1):
    tags = links.find('a', {'aria-label': 'Google Scholar reference ' +str(k)})
    if tags==None:
        paperLinksgoogleScholarspringer.append("No Google Scholar Link")
    else:
        paperLinksgoogleScholarspringer.append(tags['href'])
    k+=1
# print(paperLinksgoogleScholarspringer)



#to get the doi of the references.
springerDOI = []
refNumber = len(links)
# print(refNumber)

i=0

for i in range(1, refNumber+1):
    doiLinks = links.find('a', {'aria-label': 'Article reference '+str(i)})
    # print(doiLinks)

    if doiLinks == None:
        springerDOI.append("No DOI")
    else:
        springerDOI.append(doiLinks['href'])
    i+=1

# print(springerDOI)


#extracting the authors from the main page itself.
text = soup.find_all('ol', class_='c-article-references')

k = 0


text_main = soup.find_all('li', class_='c-article-references__item js-c-reading-companion-references-item')

Titles = []
allLastAuthors_springer = []
journalName = []
yearPublication = []

for ref in text_main:
    string = ref.find('p', class_="c-article-references__text")

    togetdeets = string.text.split(',')
    content = togetdeets[-2].split('.')
    Titles.append(content[0]) # gets the title of the references

    if len(togetdeets)<4:
        allLastAuthors_springer.append("No Data Found")
    else:
        allLastAuthors_springer.append(togetdeets[-3])

    sum = ''
    for j in range(1, len(content)-1):
        sum += content[j]
    journalName.append(sum) # gets the journal name of the references

    year = togetdeets[-1].split('(')
    # print(year[-1])

    if year[-1][0]==" ":
        yearPublication.append("No Data Found") # gets the year of publication of the references
    else:
        year_new = year[-1].split(')')
        yearPublication.append(year_new[0]) #gets the year of publication of the references


# print(len(springerDOI))

In [60]:
# extraction from "Elsevier(sciencedirect)" publication house.
# html = gettingUrl("https://www.sciencedirect.com/science/article/pii/S2376060522000608")
paperLinksgoogleScholarelsevier = []
links = []
Titles =[]
elsevierDOI = []
# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'lxml')
tags = soup.find_all('dd', class_="reference")

refNumber = len(tags)

i = 0

for i in range(1, int(refNumber)+1):
    tag = soup.find('dd', {'id': 'sref'+str(i)})
    print(tag)


    

# for i in range(1,int(refNumber)+1):
#     tag = soup.find('dd', {'id':'sref'+str(i)})
#     print(tag)

# for tag in tags:
#     link = (tag.find_all('a', class_="link"))
#     for l in link:
#         links.append(l['href'])
    
#     doi = tag.find('a', class_="link")
#     elsevierDOI.append("https://www.sciencedirect.com/"+doi['href'])
    

# for link in links:
#     if "scholar" in link:
#         paperLinksgoogleScholarelsevier.append(link)

# for link in paperLinksgoogleScholarelsevier:
#     index = paperLinksgoogleScholarelsevier.index(link)

# for tag in tags:
#     Titles.append(((tag.find("div", class_="contribution")).find('strong',class_='title')).text)



TypeError: 'list' object is not callable

In [ ]:
# extraction from "MDPI" publication house.
paperLinksgoogleScholarMDPI = []
html = gettingUrl("https://www.mdpi.com/2226-4310/9/11/704/htm")
# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'lxml')
tags = soup.find('ol', class_="html-xx")
links = tags.find_all('a', class_="google-scholar")
for link in links:
    paperLinksgoogleScholarMDPI.append(link['href'])
    index = links.index(link)
    print(index+1, link['href'])


In [ ]:
# extraction from "Science" publication house.
html = gettingUrl("https://science.sciencemag.org/content/371/6534/eaay9982")
paperLinksgoogleScholarScienceall = []
paperLinksgoogleScholarScience = []
# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'lxml')
tags = soup.find('section', {'id': 'bibliography'})
links = tags.find_all('a')
for link in links:
    paperLinksgoogleScholarScienceall.append(link['href'])

for link in paperLinksgoogleScholarScienceall:
    if "scholar" in link:
        paperLinksgoogleScholarScience.append(link)
        index = paperLinksgoogleScholarScience.index(link)
        print(index+1, link)


In [ ]:
# extraction from "IEEE" publication house.
html = gettingUrl("https://ieeexplore.ieee.org/document/9837920/references#references")
paperLinksgoogleScholarIEEE = []
paperLinksgoogleScholarIEEEfinal = []
# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'lxml')
tags = soup.find_all('div', class_='reference-container')
for tag in tags:
    link = tag.find('a', class_="stats-reference-link-googleScholar")
    paperLinksgoogleScholarIEEE.append(link['href'])

initial = paperLinksgoogleScholarIEEE[0]
paperLinksgoogleScholarIEEEfinal.append(initial)
i = 1
while (i < len(paperLinksgoogleScholarIEEE)):
    if paperLinksgoogleScholarIEEE[i] != initial:
        paperLinksgoogleScholarIEEEfinal.append(paperLinksgoogleScholarIEEE[i])
    else:
        break
    i += 1

for link in paperLinksgoogleScholarIEEEfinal:
    index = paperLinksgoogleScholarIEEEfinal.index(link)
    print(index+1, link)


In [ ]:
# extraction from "Cambridge publishing house" publication house.
html = gettingUrl("https://www.cambridge.org/core/journals/experimental-results/article/observing-nonuniform-nonluders-yielding-in-a-coldrolled-medium-manganese-steel-with-digital-image-correlation/1C8B6F3364BA54FC2C1D7801FCFDB85E")
paperlinksCambridge = []
paperlinksgoogleScholarCambridge = []

# creating an instance of the BeautifulSoup library
soup = BeautifulSoup(html, 'lxml')
tags = soup.find('div', {'id': 'references-list'})
links = tags.find_all('a', class_="ref-link")
for link in links:
    paperlinksCambridge.append(link['href'])
for link in paperlinksCambridge:
    if "scholar" in link:
        paperlinksgoogleScholarCambridge.append(link)
        index = paperlinksgoogleScholarCambridge.index(link)
        print(index+1, link)


In [ ]:
def Publication(link):
    x = re.findall('\.(.*?)\.', link)
    return x

print(Publication("https://link.springer.com/article/10.1007/s43673-022-00064-1"))
print(Publication("https://www.nature.com/articles/s41417-022-00495-w"))

In [ ]:
string = "https://link.springer.com/article/10.1007/s43673-022-00064-1"
str = string.split('.')

if "springer" in str:
    print("yes")